In [106]:
#Note: The openai-python library support for Azure OpenAI is in preview.
#This model's maximum context length is 8192 tokens
# %pip install openai
# %pip install azure.search.documents
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
import json
import os
import openai
import sys
openai.api_type = "azure"
openai.api_base = "https://openai-pulte.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
# Add below to local environment or use '../inputs/environment.py' to add to local environment (ex: os.environ['OPENAI_API_KEY'] = '')
sys.path.append(os.path.abspath('../inputs'))
import environment
openai.api_key = openai.api_key = os.getenv("OPENAI_API_KEY") 
azure_search_key = os.environ.get("AZURE_SEARCH_KEY")
azure_search_service = os.environ.get("AZURE_SEARCH_SERVICE")
azure_search_index = os.environ.get("AZURE_SEARCH_INDEX")

In [107]:
question = "What communities in Michigan having prices starting below 600,000?"

system_rules = """You are an AI assistant that helps people find the right new home.
Answer ONLY with the facts listed in the list of csv below. If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
CSV:
BrandName,CommunityName,StateName,MinBedrooms,MaxBedrooms,MinBathrooms,MaxBathrooms,StartingFromPrice,MaxBasePrice
"""

top = 10

In [108]:
credential = AzureKeyCredential(azure_search_key)
search_client = SearchClient(
    endpoint=f"https://{azure_search_service}.search.windows.net",
    index_name=azure_search_index,
    credential=credential)
search_text = '*'
search_options = {
    'filter': 'CommunityActiveFlag eq true',
    'select': 'BrandName,CommunityName,StateName,MinBedrooms,MaxBedrooms,MinBathrooms,MaxBathrooms,StartingFromPrice,MaxBasePrice',
    'top': top
}
search_results = search_client.search(search_text=search_text, **search_options)
# Lower cost to supply data as CSV versus JSON due to only having to list the header once
for doc in search_results:
    system_rules += f"{doc['BrandName']},{doc['CommunityName']},{doc['StateName']},{doc['MinBedrooms']},{doc['MaxBedrooms']},{doc['MinBathrooms']},{doc['MaxBathrooms']},{doc['StartingFromPrice']},{doc['MaxBasePrice']}\n"
print(system_rules)

You are an AI assistant that helps people find the right new home.
Answer ONLY with the facts listed in the list of csv below. If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
CSV:
BrandName,CommunityName,StateName,MinBedrooms,MaxBedrooms,MinBathrooms,MaxBathrooms,StartingFromPrice,MaxBasePrice
Pulte,Gemstone Valley,Michigan,0,6,0.0,4.0,233990.0,2660600.0
Pulte,Long Ridge & Bellewood Estates,Indiana,0,4,2.5,2.5,0.0,0.0
Pulte,Fireside at Desert Ridge,Michigan,4,4,3.5,3.5,419990.0,419990.0
Pulte,Potomac Yard,Virginia,3,3,2.5,4.5,649990.0,899035.0
Centex,Spence Creek,Tennessee,3,5,2.5,4.5,192990.0,328990.0
Centex,Magnolia Park,Florida,2,4,2.0,3.5,144990.0,224990.0
Pulte,Jonathans Landing,Massachusetts,2,2,2.0,2.0,325900.0,325900.0
Pulte,Hawthorn of Aurora,Ohio,4,4,2.5,3.5,349990.0,411990.0
Pulte,Stonehaven-The Enclave,Texas,3,5,2.0,3.5,224990.0,25

In [109]:
response = openai.ChatCompletion.create(
    engine="GPT-35",
    messages = [
        {
            "role":"system",
            "content": system_rules
        },
        {
            "role":"user",
            "content": question
        }
    ],
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None)
print(response.choices[0].message.content)
print('\nFull Returned Object:\n')
print(response)

The only community in Michigan with prices starting below 600,000 is Gemstone Valley by Pulte, with a starting price of 233,990.

Full Returned Object:

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The only community in Michigan with prices starting below 600,000 is Gemstone Valley by Pulte, with a starting price of 233,990.",
        "role": "assistant"
      }
    }
  ],
  "created": 1681495726,
  "id": "chatcmpl-75I8kWVBYvPdsAl4hDATO5cxUksfs",
  "model": "gpt-35-turbo",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 34,
    "prompt_tokens": 461,
    "total_tokens": 495
  }
}
